In [3]:
import pymongo
import pandas as pd
import matplotlib as plt
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures

market = futures.market
unit = futures.unit
position = futures.position

#加载数据
market = DataFrame(list(market.find()))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find()))

#大写字母
position['variety']=position['variety'].str.upper()

#删除/选取某行含有特殊数值的列
position=position.set_index('名次')

#选择需要显示的字段
data1=market[['date','variety','set_close']]

data2=unit[[ 'variety','unit']]
position=position[['date','variety','symbol','持买仓量期货公司','持买仓量', '持买仓量增减','持卖仓量期货公司','持卖仓量', '持卖仓量增减']]

#查询会员
members='北京金鹏'
data3=position[(position['持买仓量期货公司'] == members)]
#汇总合约
data3=data3[['date','variety','持买仓量期货公司','持买仓量']]
data3=data3.groupby(['date','variety','持买仓量期货公司'])[['持买仓量']].sum()
data4=position[(position['持卖仓量期货公司'] == members)]
data4=data4[['date','variety','持卖仓量期货公司','持卖仓量']]
data4=data4.groupby(['date','variety','持卖仓量期货公司'])[['持卖仓量']].sum()
print(data3.head())
print(data4.head())

#并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: members,axis=1)

#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['持买仓量']-x['持卖仓量'],axis=1)
#选择需要显示的字段
data5=data5[['会员简称','持买仓量','持卖仓量','净持仓']]
data5=data5.reset_index(['variety','date'])
print(data5.head())
netpostion=data5.set_index('date')

netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)

#合约价值
contractValue=pd.merge(data1,data2,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['set_close']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]
#值替换replace()
contractValue=contractValue.replace(['TA'],'PTA')
print(contractValue.head())
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

#净持仓价值
sz['净持仓价值']=sz.apply(lambda x: x['净持仓']*x['contractValue'],axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
sz
# sz['黑色板块']=sz[['I']].sum(axis=1)
# sz['化工板块']=sz[['RU','MA','V','L','PP','BU','PTA']].sum(axis=1)
# sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
# sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
# sz['黑色板块']=sz[['RB','HC','ZC','J','JM','I']].sum(axis=1)
# sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
# sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','PTA','MA','PP','JM','J','ZC','I','BU','HC','SM','SF','FU']].sum(axis=1)
# sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
# sz['商品板块']=sz.sum(axis=1)
# sz


# chemical=sz.copy()
# chemical=chemical[['RU','MA','V','L','PP','BU','PTA','商品板块','工业品','化工板块']]
# chemical['max']=chemical[['RU','MA','V','L','PP','BU','PTA']].idxmax(axis=1)
# chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
# chemical['2018-10-1':]


                              持买仓量
date       variety 持买仓量期货公司       
2018-01-02 B       北京金鹏       13.0
           I       北京金鹏      150.0
2018-01-03 B       北京金鹏       14.0
           I       北京金鹏      150.0
2018-01-04 I       北京金鹏      150.0
                              持卖仓量
date       variety 持卖仓量期货公司       
2018-01-02 CS      北京金鹏      406.0
           I       北京金鹏      150.0
           JD      北京金鹏      263.0
2018-01-03 CS      北京金鹏      391.0
           I       北京金鹏      150.0
         date variety  会员简称   持买仓量   持卖仓量   净持仓
0  2018-01-02       B  北京金鹏   13.0    0.0  13.0
1  2018-01-02       I  北京金鹏  150.0  150.0   0.0
2  2018-01-03       B  北京金鹏   14.0    0.0  14.0
3  2018-01-03       I  北京金鹏  150.0  150.0   0.0
4  2018-01-04       I  北京金鹏  150.0  150.0   0.0
         date variety  contractValue
0  2018-05-04       A        38330.0
1  2018-05-04      AG        55380.0
2  2018-05-04      AL        73100.0
3  2018-05-04      AP        82150.0
4  2018-05-04      AU       272000.0


,variety,A,B,C,CS,I,JD,M,P,Y
date,会员简称,,,,,,,,,
2018-01-02,北京金鹏,0,425620,0,-8692460,0,-10238590,0,0,0
2018-01-03,北京金鹏,0,459200,0,-8465150,0,-10183950,0,0,0
2018-01-04,北京金鹏,-36530,0,0,0,0,-10152150,0,0,0
2018-01-05,北京金鹏,0,0,0,0,0,-10046600,0,0,0
2018-01-08,北京金鹏,0,0,0,-6805200,0,-9910560,0,0,0
2018-01-09,北京金鹏,0,0,0,0,28100000,-9877400,0,0,0
2018-01-10,北京金鹏,0,0,0,-6165400,27900000,-9848580,0,0,0
2018-01-11,北京金鹏,-36530,0,0,0,27850000,-9840720,0,0,0
2018-01-12,北京金鹏,0,0,0,0,27304500,-9838100,0,0,0


In [11]:
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','J','JM','I','黑色板块']] 
黑色板块['max']=黑色板块[['RB','HC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','J','JM','I']].idxmin(axis=1)
黑色板块

,variety,RB,HC,J,JM,I,黑色板块,max,min
date,会员简称,,,,,,,,
2018-01-02,道通期货,0,5883000,20150000,0,-322324800,-296291800,J,I
2018-01-03,道通期货,-843277220,6948000,20150000,0,-328889600,-1145068820,J,RB
2018-01-04,道通期货,-919657470,6969600,0,0,-270848400,-1183536270,HC,RB
2018-01-05,道通期货,-929133280,6908400,0,0,-255162600,-1177387480,HC,RB
2018-01-08,道通期货,-727558320,0,0,0,-240303200,-967861520,HC,RB
2018-01-09,道通期货,-725003820,0,0,0,-243177400,-968181220,HC,RB
2018-01-10,道通期货,-733012360,0,0,0,-246189600,-979201960,HC,RB
2018-01-11,道通期货,0,0,0,0,-284627000,-284627000,RB,I
2018-01-12,道通期货,0,0,0,0,-219907500,-219907500,RB,I
